In [1]:
#!pip install nltk

In [2]:
import pandas as pd
import numpy as np

from numpy.random import seed
#from tensorflow import set_random_seed
import tensorflow as tf
import random as rn
import os
from nltk import tokenize 
import nltk
from sklearn.model_selection import StratifiedKFold, cross_val_score, train_test_split, KFold
nltk.download('punkt')
from keras.layers import Embedding, SpatialDropout1D
from keras.layers import Dense, Input, GRU, LSTM 
from keras.layers import Bidirectional, Dropout, GlobalMaxPool1D 
from keras.layers import LSTM, GRU, GlobalAveragePooling1D
from keras.layers import Conv1D, GlobalMaxPooling1D, TimeDistributed
from keras.layers import Dense, Embedding, Input
import keras

from keras.models import Model, Sequential
from keras.optimizers import RMSprop
import keras.backend as K
from keras.engine.topology import Layer, InputSpec
from keras.models import load_model

from keras.preprocessing import text, sequence
from keras import initializers as initializers, regularizers, constraints

from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score, average_precision_score
from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn.preprocessing import LabelEncoder, LabelBinarizer

[nltk_data] Downloading package punkt to
[nltk_data]     /home/RDC/zinovyee.hub/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [3]:
data = pd.read_csv("dapps_with_features_v4.csv")

In [4]:
RS = 42
SPLITS = 5
LENGTH = 3420 

In [5]:
X = data.drop('category', axis=1)
y = data.category

In [6]:
X, X_test, y, y_test  = train_test_split(X, y, stratify=y, test_size=0.2, random_state=RS)

In [7]:
print(f'X_train shape {X.shape}')
print(f'X_test shape {X_test.shape}')

X_train shape (1716, 17)
X_test shape (430, 17)


# Training on full code

## Check for possible meta-parameters

In [10]:
if tf.test.is_gpu_available():
  import models
else:
  import models_no_gpu as models

Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.


[nltk_data] Downloading package punkt to
[nltk_data]     /home/RDC/zinovyee.hub/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [11]:
names = [f'aucroc_{i}' for i in ['development', 'exchanges', 'finance', 'gambling', 'games',
       'high-risk', 'marketplaces', 'media', 'other', 'social']]
names2 = [f'aucprc_{i}' for i in ['development', 'exchanges', 'finance', 'gambling', 'games',
       'high-risk', 'marketplaces', 'media', 'other', 'social']]
names.extend(names2)

In [12]:
kf  = StratifiedKFold(n_splits=SPLITS, random_state=RS)

/home/RDC/zinovyee.hub/anaconda3/envs/dmc/lib/python3.8/site-packages/sklearn/model_selection/_split.py:292: FutureWarning: Setting a random_state has no effect since shuffle is False. This will raise an error in 0.24. You should leave random_state to its default (None), or set shuffle=True.
  warnings.warn(


In [14]:
metrics_cv = pd.DataFrame(np.zeros((SPLITS, len(names))), columns=names)
for c, (train_index, val_index) in enumerate(kf.split(X, y)):

        X_train, X_val      = X.iloc[train_index]['SC_full'], X.iloc[val_index]['SC_full']
        y_train, y_val      = y.iloc[train_index], y.iloc[val_index] 
        
        tokenizer = keras.preprocessing.text.Tokenizer(num_words=40000)
        tokenizer.fit_on_texts(X_train)
        
        list_tokenized_train = tokenizer.texts_to_sequences(X_train)
        list_tokenized_val   = tokenizer.texts_to_sequences(X_val)

        X_train = sequence.pad_sequences(list_tokenized_train, maxlen=LENGTH)
        X_val   = sequence.pad_sequences(list_tokenized_val, maxlen=LENGTH)

        le = LabelBinarizer()
        le.fit(y_train)

        y_train = le.transform(y_train)
        y_val = le.transform(y_val)

        classifier = models.dl_model(model_type='BGRU', max_features=40000, embed_dim=50, rec_units=150, dropout_rate=0.25, maxlen=LENGTH)
        classifier.fit(X_train, y_train, batch_size=32, epochs=4, shuffle=True, verbose=1)
        probs      = classifier.predict(X_val)
        
        for i, cl in enumerate(le.classes_):
            auc_roc_class = roc_auc_score(y_val[:,i], probs[:,i])
            auc_pr_class = average_precision_score(y_val[:,i], probs[:,i])

            metrics_cv.loc[c, f'aucroc_{cl}'] = round(auc_roc_class, 3)
            metrics_cv.loc[c, f'aucprc_{cl}'] = round(auc_pr_class, 3)   

print(f"AUC ROC: {round(metrics_cv.loc[:, metrics_cv.columns.str.contains('aucroc')].mean(axis=0).mean(), 3)}")
print(f"AUC PRC: {round(metrics_cv.loc[:, metrics_cv.columns.str.contains('aucprc')].mean(axis=0).mean(), 3)}")

Epoch 1/4
43/43 [==============================] - 297s 7s/step - loss: 2.0120 - acc: 0.3163
Epoch 2/4
43/43 [==============================] - 295s 7s/step - loss: 1.9074 - acc: 0.3695
Epoch 3/4
43/43 [==============================] - 291s 7s/step - loss: 1.7989 - acc: 0.3994
Epoch 4/4
43/43 [==============================] - 294s 7s/step - loss: 1.7423 - acc: 0.4242
Epoch 1/4
43/43 [==============================] - 297s 7s/step - loss: 2.0352 - acc: 0.3168
Epoch 2/4
43/43 [==============================] - 288s 7s/step - loss: 1.9198 - acc: 0.3714
Epoch 3/4
43/43 [==============================] - 270s 6s/step - loss: 1.8654 - acc: 0.3904
Epoch 4/4
43/43 [==============================] - 213s 5s/step - loss: 1.8349 - acc: 0.4151
Epoch 1/4
43/43 [==============================] - 187s 4s/step - loss: 2.0063 - acc: 0.3052
Epoch 2/4
43/43 [==============================] - 190s 4s/step - loss: 1.8748 - acc: 0.3795
Epoch 3/4
43/43 [==============================] - 192s 4s/step - loss

NameError: name 'c_p' is not defined

In [15]:
print(f"AUC ROC: {round(metrics_cv.loc[:, metrics_cv.columns.str.contains('aucroc')].mean(axis=0).mean(), 3)}")
print(f"AUC PRC: {round(metrics_cv.loc[:, metrics_cv.columns.str.contains('aucprc')].mean(axis=0).mean(), 3)}")

AUC ROC: 0.697
AUC PRC: 0.268


In [16]:
round(metrics_cv.loc[:, metrics_cv.columns.str.contains('aucroc')].mean(axis=0), 3)

aucroc_development     0.648
aucroc_exchanges       0.870
aucroc_finance         0.719
aucroc_gambling        0.709
aucroc_games           0.732
aucroc_high-risk       0.723
aucroc_marketplaces    0.719
aucroc_media           0.599
aucroc_other           0.615
aucroc_social          0.633
dtype: float64

In [17]:
round(metrics_cv.loc[:, metrics_cv.columns.str.contains('aucprc')].mean(axis=0), 3)

aucprc_development     0.058
aucprc_exchanges       0.809
aucprc_finance         0.369
aucprc_gambling        0.337
aucprc_games           0.491
aucprc_high-risk       0.091
aucprc_marketplaces    0.302
aucprc_media           0.055
aucprc_other           0.108
aucprc_social          0.059
dtype: float64

In [18]:
metrics_cv

,aucroc_development,aucroc_exchanges,aucroc_finance,aucroc_gambling,aucroc_games,aucroc_high-risk,aucroc_marketplaces,aucroc_media,aucroc_other,aucroc_social,aucprc_development,aucprc_exchanges,aucprc_finance,aucprc_gambling,aucprc_games,aucprc_high-risk,aucprc_marketplaces,aucprc_media,aucprc_other,aucprc_social
0,0.621,0.866,0.730,0.748,0.733,0.782,0.747,0.668,0.507,0.607,0.047,0.797,0.378,0.438,0.466,0.106,0.336,0.091,0.067,0.046
1,0.679,0.872,0.680,0.741,0.621,0.674,0.746,0.521,0.650,0.637,0.048,0.810,0.351,0.260,0.354,0.079,0.341,0.034,0.099,0.071
2,0.558,0.831,0.814,0.757,0.782,0.738,0.671,0.561,0.648,0.562,0.056,0.775,0.528,0.371,0.540,0.095,0.239,0.038,0.116,0.042
3,0.706,0.891,0.686,0.661,0.791,0.725,0.649,0.569,0.689,0.659,0.083,0.834,0.278,0.356,0.592,0.087,0.302,0.043,0.171,0.055
4,0.675,0.889,0.687,0.640,0.733,0.698,0.780,0.676,0.582,0.702,0.054,0.828,0.308,0.259,0.505,0.089,0.292,0.068,0.088,0.083


In [20]:
# TRAIN ON WHOLE DATA AND PREDICT ON TEST
metrics = pd.DataFrame(np.zeros((1,len(names))), columns=names)
tokenizer = keras.preprocessing.text.Tokenizer(num_words=40000)
tokenizer.fit_on_texts(X['SC_full'])

list_tokenized_train = tokenizer.texts_to_sequences(X['SC_full'])
list_tokenized_test   = tokenizer.texts_to_sequences(X_test['SC_full'])

X_train = sequence.pad_sequences(list_tokenized_train, maxlen=LENGTH)
X_test   = sequence.pad_sequences(list_tokenized_test, maxlen=LENGTH)

le = LabelBinarizer()
le.fit(y)

y_train = le.transform(y)
y_test_lb = le.transform(y_test)

classifier = models.dl_model(model_type='BGRU', max_features=40000, embed_dim=50, rec_units=150, dropout_rate=0.25, maxlen=LENGTH)
classifier.fit(X_train, y_train, batch_size=32, epochs=4, shuffle=True, verbose=1)
probs      = classifier.predict(X_test)

for i, cl in enumerate(le.classes_):
    auc_roc_class = roc_auc_score(y_test_lb[:,i], probs[:,i])
    auc_pr_class = average_precision_score(y_test_lb[:,i], probs[:,i])

    metrics.loc[0, f'aucroc_{cl}'] = round(auc_roc_class, 3)
    metrics.loc[0, f'aucprc_{cl}'] = round(auc_pr_class, 3)   

Epoch 1/4
54/54 [==============================] - 214s 4s/step - loss: 2.0067 - acc: 0.3246
Epoch 2/4
54/54 [==============================] - 209s 4s/step - loss: 1.9163 - acc: 0.3712
Epoch 3/4
54/54 [==============================] - 207s 4s/step - loss: 1.8682 - acc: 0.3980
Epoch 4/4
54/54 [==============================] - 205s 4s/step - loss: 1.7475 - acc: 0.4196


In [21]:
metrics

,aucroc_development,aucroc_exchanges,aucroc_finance,aucroc_gambling,aucroc_games,aucroc_high-risk,aucroc_marketplaces,aucroc_media,aucroc_other,aucroc_social,aucprc_development,aucprc_exchanges,aucprc_finance,aucprc_gambling,aucprc_games,aucprc_high-risk,aucprc_marketplaces,aucprc_media,aucprc_other,aucprc_social
0,0.706,0.85,0.656,0.657,0.764,0.734,0.718,0.689,0.66,0.694,0.065,0.796,0.289,0.297,0.539,0.116,0.341,0.082,0.159,0.071


In [22]:
round(metrics.loc[:, metrics.columns.str.contains('aucroc')].mean(axis=1), 3)

0    0.713
dtype: float64

In [23]:
round(metrics.loc[:, metrics.columns.str.contains('aucprc')].mean(axis=1), 3)

0    0.276
dtype: float64